In [1]:
import pandas as pd
import numpy as np

In [2]:
# Parameters
spot_filename:str = "/tmp/index.csv"
options_filename:str = "data/nifty_options_eod.h5"
option_type:str = 'PE'
start_day:int = 4
de:int = 0
step:int = 100 
stop:float = 0.02

In [3]:
index = pd.read_csv(spot_filename, parse_dates=['date'])
opt = pd.read_hdf(options_filename).sort_values(by=['date'])
rename = {
    'open_price': 'open',
    'high_price': 'high',
    'low_price': 'low',
    'close_pric': 'close'
}
opt = opt.rename(columns=rename)
opt['de'] = (opt['expiry_date'] - opt['date']).dt.days

In [4]:
ce = opt.query("opt_type=='CE'")
pe = opt.query("opt_type=='PE'")

In [27]:
def get_result(options_data, spot_data, wkday=4, de=0, itm=0, step=100, holding_days=7):
    weekly = spot_data.query(f"wkday=={wkday}").copy()
    weekly['strike'] = [int(x/step)*step for x in weekly.close.values]
    opts = options_data.merge(weekly[['date', 'close', 'strike']].rename(
        columns={'close':'spot'}), on=['date', 'strike']).copy()
    opts = opts.query(f"de>{de}")
    opts = opts.sort_values(by='de').groupby('date').first().reset_index()
    cols = ['date', 'contract_d', 'strike', 'expiry_date', 'close', 'spot']
    entries = opts[cols].copy()
    entries['entry_date'] = entries.date.copy()
    exit_frame = entries.copy()
    exit_frame['date'] = exit_frame.date + pd.DateOffset(days=holding_days)
    del exit_frame['close'] # Since we need the other close in expiry
    del exit_frame['contract_d'] 
    opt2 = exit_frame.merge(options_data, on=['date', 'strike', 'expiry_date'])
    exits = opt2[cols + ['entry_date']]
    exits = exits.rename(columns={'date':'exit_date', 'entry_date': 'date'})
    trades = entries.merge(exits, on=['strike', 'expiry_date', 'contract_d', 'date'])
    rename = {
        'date_x': 'entry_date',
        'close_x': 'entry_price',
        'spot_x': 'entry_spot',
        'date_y': 'exit_date',
        'close_y': 'exit_price',
        'spot_y': 'exit_spot'
    }
    trades = trades.rename(columns=rename)
    trades['pnl'] = trades.eval('exit_price-entry_price')
    trades['de'] = (trades['exit_date'] - trades['entry_date']).dt.days
    return trades

res = get_result(pe, index)
res.pnl.describe()

count     255.000000
mean      -12.007451
std       157.089709
min      -451.950000
25%       -69.100000
50%       -40.050000
75%        -1.650000
max      1504.800000
Name: pnl, dtype: float64